This notebook shows you how to create and query a table or DataFrame loaded from data stored in Azure Blob storage.

### Step 1: Set the data location and type

There are two ways to access Azure Blob storage: account keys and shared access signatures (SAS).

To get started, we need to set the location and type of the file.

In [0]:
storage_account_name = "teststoragedatabricks2"
storage_account_access_key = "OEMTXV8wTdSRZhDo2ibwtEdicrhkLbIqneXu+y/fHA1bBDQmRUsAzWWgSiQDW069vrjgwPDDnsqX+ASt/zNHBQ=="

In [0]:
file_location = "wasbs://databrickscontainer@teststoragedatabricks2.blob.core.windows.net/"
file_type = "csv"

In [0]:
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

### Step 2: Read the data

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.

In [0]:
df = spark.read.format(file_type).option("header", "true").load(file_location)

### Step 3: Query the data

Now that we have created our DataFrame, we can query it. For instance, you can identify particular columns to select and display.

In [0]:
display(df.select("Name"))

Name
David
Malcolm


### Step 4: (Optional) Create a view or table

If you want to query this data as a table, you can simply register it as a *view* or a table.

In [0]:
df.createOrReplaceTempView("temp_view")

We can query this view using Spark SQL. For instance, we can perform a simple aggregation. Notice how we can use `%sql` to query the view from SQL.

In [0]:
%sql

SELECT Name, Age FROM temp_view 

Name,Age
David,27
Malcolm,56


Since this table is registered as a temp view, it will be available only to this notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.

In [0]:
df.write.format("parquet").saveAsTable("TEST")

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-4453212634070679>, line 1
----> 1 df.overwrite.format("parquet").saveAsTable("TEST")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3068, in DataFrame.__getattr__(self, name)
   3035 """Returns the :class:`Column` denoted by ``name``.
   3036 
   3037 .. versionadded:: 1.3.0
   (...)
   3065 +---+
   3066 """
   3067 if name not in self.columns:
-> 3068     raise AttributeError(
   3069         "'%s' object has no attribute '%s'" % (self.__class__.

This table will persist across cluster restarts and allow various users across different notebooks to query this data.

In [0]:
dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4159197055266531>, line 2
      1 dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
----> 2 cluster_id = notebook_content.get("metadata", {}).get("cluster_id", None)

NameError: name 'notebook_content' is not defined